In [ ]:
# CELL 1 – Imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import joblib
import os

print("Study plan generator imports successful!")


In [ ]:
# CELL 2 – Create study plan templates
study_plan_templates = {
    "Python basics": {
        "easy": [
            "Introduction to Python syntax (variables, types)",
            "Control flow (if, for, while loops)",
            "Functions and scope",
            "Lists, dictionaries, and tuples",
            "File I/O basics",
        ],
        "medium": [
            "Object-oriented programming (classes, inheritance)",
            "Exception handling",
            "Decorators and generators",
            "Working with NumPy arrays",
            "Introduction to pandas",
        ],
    },
    "Machine Learning basics": {
        "easy": [
            "What is machine learning?",
            "Supervised vs unsupervised learning",
            "Train/test split concept",
            "Overfitting and underfitting",
            "Basic metrics (accuracy, precision, recall)",
        ],
        "medium": [
            "Linear regression deep dive",
            "Logistic regression for classification",
            "Decision trees and random forests",
            "K-means clustering",
            "Feature engineering and selection",
        ],
    },
    "Deep Learning basics": {
        "easy": [
            "Neural network fundamentals",
            "Activation functions",
            "Backpropagation concept",
            "Building a simple network",
            "Introduction to Keras",
        ],
        "medium": [
            "Convolutional Neural Networks (CNNs)",
            "Recurrent Neural Networks (RNNs)",
            "Transfer learning",
            "Hyperparameter tuning",
            "Regularization techniques",
        ],
    },
}

print("Study plan templates loaded")
for subject, levels in study_plan_templates.items():
    print(f"  {subject}: {list(levels.keys())}")


In [ ]:
# CELL 3 – Define study plan generator
def generate_study_plan(subject: str, total_hours: int, difficulty: str = "medium") -> dict:
    """
    Generate a structured study plan.
    
    Args:
        subject: Topic to study
        total_hours: Total hours available for study
        difficulty: "easy" or "medium"
    
    Returns:
        Dictionary with daily study breakdown
    """
    if subject not in study_plan_templates:
        return {"error": f"Subject '{subject}' not found"}
    
    topics = study_plan_templates[subject].get(difficulty, [])
    num_days = max(3, min(7, len(topics)))  # 3-7 days
    
    hours_per_day = total_hours / num_days
    minutes_per_topic = int((hours_per_day * 60) / (len(topics) / num_days))
    
    plan = {
        "subject": subject,
        "difficulty": difficulty,
        "total_hours": total_hours,
        "num_days": num_days,
        "hours_per_day": round(hours_per_day, 1),
        "daily_schedule": []
    }
    
    # Distribute topics across days
    topics_per_day = len(topics) / num_days
    for day in range(num_days):
        start_idx = int(day * topics_per_day)
        end_idx = int((day + 1) * topics_per_day)
        day_topics = topics[start_idx:end_idx]
        
        plan["daily_schedule"].append({
            "day": day + 1,
            "topics": day_topics,
            "minutes_per_topic": minutes_per_topic,
            "total_minutes": len(day_topics) * minutes_per_topic,
        })
    
    return plan

# Test
plan = generate_study_plan("Python basics", total_hours=10, difficulty="easy")
print("\nGenerated Study Plan:")
print(f"Subject: {plan['subject']}")
print(f"Total hours: {plan['total_hours']}, Spread over {plan['num_days']} days")
for day in plan["daily_schedule"]:
    print(f"\nDay {day['day']}:")
    for topic in day["topics"]:
        print(f"  - {topic} ({day['minutes_per_topic']} min)")


In [ ]:
# CELL 4 – Convert plan to CSV (for download)
def study_plan_to_csv(plan: dict) -> str:
    """
    Convert study plan to CSV format string.
    """
    lines = [
        f"Study Plan: {plan['subject']}",
        f"Difficulty: {plan['difficulty']}",
        f"Total Hours: {plan['total_hours']}",
        f"Days: {plan['num_days']}",
        "",
        "Day,Topics,Minutes"
    ]
    
    for day in plan["daily_schedule"]:
        day_num = day["day"]
        topics_str = " | ".join(day["topics"])
        total_min = day["total_minutes"]
        lines.append(f"{day_num},\"{topics_str}\",{total_min}")
    
    return "\n".join(lines)

# Test
csv_output = study_plan_to_csv(plan)
print("\nCSV Output:")
print(csv_output)


In [ ]:
# CELL 5 – Save study plan components
models_dir = "../backend/models"
os.makedirs(models_dir, exist_ok=True)

plan_path = os.path.join(models_dir, "study_plan_templates.joblib")
joblib.dump(study_plan_templates, plan_path)

print(f"Study plan templates saved to: {plan_path}")
